# Data cleaning

In this notebook we will clean our Dataframe 'Kickstarter_merged.csv'. That means we will create new columns which we think are important as well as we will drop columns with unimportance.

#### Overview new columns
    * column 'blurbs' will be replaced with -> 'blurb_len_c', 'blurb_len_w'
    * column 'slug' will be trplaced with -> 'slug_len_c'. 'slug_len_w', 'cat_in_slug'
    * column 'category' will be replaced with -> 'category_parent_id', 'category_id', 'category_name'
    * column 'created_at' will be replaced with -> 'created_year', 'created_month'
    * column 'deadline' will be replaced with -> 'deadline_year', 'deadline_month'
    * column 'goal' will be converted in USD
    * column 'launched_at' will be replaced with -> 'launched_year', 'launched_month'
    * new column duration of the crowdfunding
    * new column preparation time (from created_at until launched_at
    * column 'state_changed_at' will be replaced with -> 'state_changed_year', 'state_changed_month, 'state_changed_at_day' ???
    * new column pledged/backer
    * new column pledged/goal in %


#### Overview dropped columns
    * converted_pledged_amount
    * creator
    * currency 
    * currency_symbol
    * currency_trailing_code
    * current_curency
    * disable_communication
    * friends
    * fx_rate
    * id after using it for other transformations
    * is_backing
    * is_starrable
    * is_starred
    * location
    * name 
    * permissions
    * photo
    * pledged
    * profile 
    * source_url
    * spotlight
    * state_changed_at ??
    * static_usd_rate 
    * urls
    * usd_type
    
    
#### Overview dropped rows
    * 8 rows with missing values in column 'blurbs'
    * drop the duplicates
    * drop rows with values 'suspended' and 'live' in column 'state'

In [1]:
# import packages
import pandas as pd
import numpy as np
import seaborn as sns
import time
import datetime as dt
import json

Read in file.

In [2]:
df = pd.read_csv('data/Kickstarter_merged.csv', index_col=0)

## Create new columns
    
Let´s start with creating new columns: We will replace the columns 'blurbs', 'slug', category', 'created_at', 'deadline', 'launched_at', 'state_changed_at'.

In [3]:
#function to return length of string (number of characters)
def string_len_c(string):
    string_str = str(string)
    string_len = len(string_str)
    return string_len

In [4]:
#function to return length of string (number of words)
def string_len_w(string):
    string_str = str(string)
    string_list = string_str.split()
    string_len = len(string_list)
    return string_len

In [5]:
#function to return length of string (number of words, seperated by "-"
def string_len_slug_w(string):
    string_str = str(string)
    string_list = string_str.split("-")
    string_len = len(string_list)
    return string_len

In [6]:
#function that checks if category or subcategory in slug
def word_in_lists(string, list_cat, list_subcat):
    string_str = str(string)
    string_list = string_str.split("-")
    for word in string_list:
        if word in list_cat:
            return word
        elif word in list_subcat:
            return word
        else:
            return 0

In [7]:
# replace column 'blurbs' and 'slug'
df['blurb_len_c'] = df.apply(lambda x: string_len_c(x['blurb']), axis=1)
df['blurb_len_w'] = df.apply(lambda x: string_len_w(x['blurb']), axis=1)

df['slug_len_c'] = df.apply(lambda x: string_len_c(x['slug']), axis=1)
df['slug_len_w'] = df.apply(lambda x: string_len_slug_w(x['slug']), axis=1)

In [8]:
subcategories = ['Playing Cards', 'Rock', 'Nonfiction', 'Classical Music', 'Music',
       'Restaurants', 'Small Batch', 'Footwear', 'Sculpture',
       'Photography', 'Fashion', 'Mixed Media', 'Software',
       'Performance Art', 'Theater', 'Graphic Design', 'Narrative Film',
       'Television', 'Sound', 'Video Art', 'Printing', 'Anthologies',
       'Art Books', 'Live Games', 'Performances', 'Technology',
       'Civic Design', 'Literary Spaces', 'Taxidermy', 'Vegan', 'Gadgets',
       'Spaces', 'Pet Fashion', 'Poetry', 'Jewelry', 'Puzzles',
       'Camera Equipment', 'Installations', '3D Printing', 'Indie Rock',
       'Events', 'Accessories', 'Shorts', 'Documentary', 'Web',
       'Translations', 'Comics', 'Pottery', 'Tabletop Games', 'Art',
       'Faith', 'Zines', 'Action', 'Public Art', 'Ceramics',
       'Illustration', 'Hip-Hop', 'Cookbooks', 'Conceptual Art',
       'Wearables', 'Food', 'Thrillers', 'Design', 'World Music',
       'Movie Theaters', "Children's Books", 'Academic', 'Video Games',
       'Childrenswear', 'Music Videos', 'Fantasy', 'Product Design',
       'Latin', 'Romance', 'Stationery', 'Typography', 'Comedy',
       'Apparel', 'Animation', 'Fine Art', 'Woodworking', 'Experimental',
       'Digital Art', 'DIY', 'Crafts', 'Gaming Hardware', 'Glass',
       'Painting', 'Architecture', 'Young Adult', 'R&B', 'People',
       'Film & Video', 'Robots', 'Blues', 'Periodicals', 'Kids',
       'Candles', 'Knitting', 'Workshops', 'Jazz', 'Pop', 'Dance',
       'Fiction', 'Family', 'Flight', 'Country & Folk', 'Science Fiction',
       'Interactive Design', 'Plays', 'Calendars', 'Print', 'Weaving',
       'Comic Books', 'Webseries', 'Animals', 'Photobooks', 'Food Trucks',
       'Apps', 'Drinks', 'Graphic Novels', 'Webcomics',
       'Fabrication Tools', 'Textiles', 'Publishing', 'Ready-to-wear',
       'Audio', 'Horror', 'Nature', 'Video', 'Makerspaces',
       "Farmer's Markets", 'Punk', 'Literary Journals', 'Bacon',
       'Community Gardens', 'Letterpress', 'Drama', 'Mobile Games',
       'Hardware', 'Farms', 'Places', 'Space Exploration', 'Journalism',
       'Musical', 'Festivals', 'Radio & Podcasts', 'Couture', 'Metal',
       'Electronic Music', 'DIY Electronics', 'Games', 'Immersive',
       'Residencies', 'Embroidery', 'Photo', 'Quilts', 'Crochet',
       'Chiptune']

In [9]:
# making lower case subcategories and split if more than one word
subcategories_lower = []
for cat in subcategories:
    cat_split = cat.split()
    for cat in cat_split:
        subcategories_lower.append(cat.lower())

In [10]:
categories = ['Games', 'Music', 'Publishing', 'Food', 'Fashion', 'Art',
       'Photography', 'Technology', 'Theater', 'Design', 'Film & Video',
       'Crafts', 'Comics', 'Dance', 'Journalism']

In [11]:
# making lower case subcategories and split if more than one word
categories_lower = []
for cat in categories:
    cat_split = cat.split(" & ")
    for cat in cat_split:
        categories_lower.append(cat.lower())

In [12]:
df['cat_in_slug'] = df.apply(lambda x: word_in_lists(x['slug'], categories_lower, subcategories_lower), axis=1)

In [13]:
# new columns with information from 'category'
df['category_parent_id'] = pd.DataFrame([json.loads(df["category"][i]).get("parent_id") for i in range(df.shape[0])])
df['category_id'] = pd.DataFrame([json.loads(df["category"][i]).get("id") for i in range(df.shape[0])])
df['category_name'] = pd.DataFrame([json.loads(df["category"][i]).get("name") for i in range(df.shape[0])])
df['category_parent_id'] = df['category_parent_id'].fillna(0).astype("int")

In [14]:
#funtion to extract the year out of the number
def extract_year(number):
    gmtime = time.gmtime(number)
    return gmtime[0]

In [15]:
#funtion to extract the month out of the number
def extract_month(number):
    gmtime = time.gmtime(number)
    return gmtime[1]

In [16]:
#funtion to extract the month out of the number
def extract_day(number):
    gmtime = time.gmtime(number)
    return gmtime[2]

In [17]:
# replace column 'created_at'
df["created_year"] = df.apply(lambda x: extract_year(x["created_at"]), axis=1)  
df["created_month"] = df.apply(lambda x: extract_month(x["created_at"]), axis=1)

In [18]:
# replace column 'deadline'
df["deadline_year"] = df.apply(lambda x: extract_year(x["deadline"]), axis=1)  
df["deadline_month"] = df.apply(lambda x: extract_month(x["deadline"]), axis=1)

In [19]:
# replace column 'launched_at'
df["launched_year"] = df.apply(lambda x: extract_year(x["launched_at"]), axis=1)  
df["launched_month"] = df.apply(lambda x: extract_month(x["launched_at"]), axis=1)

In [20]:
# function for calculating difference launched_at und deadline and converting to days
def duration(deadline, launched_at):
    duration = deadline - launched_at
    duration_complete = dt.timedelta(seconds=duration)
    return duration_complete.days

In [21]:
# calculating duration column
df["duration_days"] = df.apply(lambda x: duration(x["deadline"], x["launched_at"]), axis=1)

In [22]:
# created_at < launched at < deadline
def preparation(created_at, launched_at):
    duration = launched_at - created_at
    duration_complete = dt.timedelta(seconds=duration) 
    return duration_complete.days

In [23]:
# calculating preparation column
df["preparation"] = df.apply(lambda x: preparation(x["created_at"], x["launched_at"]), axis=1)

In [24]:
# replace column 'state_changed_at'
df["state_changed_year"] = df.apply(lambda x: extract_year(x["state_changed_at"]), axis=1)  
df["state_changed_month"] = df.apply(lambda x: extract_month(x["state_changed_at"]), axis=1)
df["state_changed_day"] = df.apply(lambda x: extract_day(x["state_changed_at"]), axis=1)

In [25]:
# create new column pledge per backer
df['pledged_per_backer'] = (df['pledged'] / df['backers_count']).round(2)

In [26]:
# create new column pledged/goal in %
df['rel_pledged_goal'] = ((df['pledged'] / df['goal']) * 100).round(2) 

The column 'goal' will also be converted into USD with the static_usd_rate to ensure the comparability.

In [27]:
# convert 'goal' in USD
df['goal'] = (df['goal'] * df['static_usd_rate']).round(2)

# round usd_pledged
df['usd_pledged'] = df['usd_pledged'].round(2)

## Drop rows

So far we created new columns, we dropped unimportant columns. Now we can drop unimportant rows because they have missing values or they don´t support our business case.

In [28]:
# drop 8 rows with missing values in column 'blurbs'
df.dropna(subset = ['blurb'], inplace=True)

In [29]:
# drop the duplicates

# creating dataframe with only the the last of all duplicates entry of the duplicates. 
df_duplicates = df[df.id.duplicated(keep='last')]

# creating dataframe and dropping all duplicates and keep the last ("newest") duplicate 
df = df.drop_duplicates(subset=['id'], keep='last')

In [30]:
# drop rows with values 'suspended' and 'live' in column 'state'
df = df.drop(df[df['state'] == "suspended" ].index)
df = df.drop(df[df['state'] == "live" ].index)

## Drop columns

Now we drop the columns we don´t need.
At first we will drop friends, is_backing, is_starred and permissions because there are a lot of missing values.

In [31]:
# drop 'friends', 'is_backing', 'is_starred', 'permissions'
df.drop(['friends', 'is_backing', 'is_starred', 'permissions'], axis=1, inplace=True)

Let´s drop the currency related columns we don´t need because we will focus on the goal and pledge amount. 

In [32]:
# drop all currency related columns 
df.drop(['converted_pledged_amount', 'currency', 'currency_symbol', 'currency_trailing_code', 'current_currency', 
         'fx_rate', 'pledged', 'static_usd_rate', 'usd_type'], axis=1, inplace = True)

We will also drop the columns which we modified/will be replaced or we needed for modifying other columns. 

In [33]:
df.drop(['blurb', 'category', 'created_at', 'deadline', 'launched_at', 'state_changed_at', 'id'], axis=1, inplace=True)

At last we will drop the other columns which are not important for our analysis. 

In [34]:
df.drop(['creator', 'disable_communication', 'is_starrable', 'location', 'name', 'photo', 'profile', 'source_url',
        'spotlight', 'urls'], axis=1, inplace=True)

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 177594 entries, 1 to 209221
Data columns (total 28 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   backers_count        177594 non-null  int64  
 1   country              177594 non-null  object 
 2   goal                 177594 non-null  float64
 3   slug                 177594 non-null  object 
 4   staff_pick           177594 non-null  bool   
 5   state                177594 non-null  object 
 6   usd_pledged          177594 non-null  float64
 7   blurb_len_c          177594 non-null  int64  
 8   blurb_len_w          177594 non-null  int64  
 9   slug_len_c           177594 non-null  int64  
 10  slug_len_w           177594 non-null  int64  
 11  cat_in_slug          177594 non-null  object 
 12  category_parent_id   177594 non-null  int64  
 13  category_id          177594 non-null  int64  
 14  category_name        177594 non-null  object 
 15  created_year     

In [36]:
df.head()

,backers_count,country,goal,slug,staff_pick,state,usd_pledged,blurb_len_c,blurb_len_w,slug_len_c,...,deadline_month,launched_year,launched_month,duration_days,preparation,state_changed_year,state_changed_month,state_changed_day,pledged_per_backer,rel_pledged_goal
1,47,US,1000.00,the-ofrenda-oracle-deck,False,successful,1950.00,125,22,23,...,9,2017,8,30,8,2017,9,9,41.49,195.00
2,271,US,15000.00,record-electras-debut-album-pop-rock-classical,False,successful,22404.00,108,15,46,...,6,2013,5,30,224,2013,6,12,82.67,149.36
3,3,GB,12160.66,the-mist-of-tribunal-a-card-game,False,failed,165.38,133,23,32,...,3,2017,1,59,5,2017,3,13,45.33,1.36
4,3,US,2800.00,help-change-the-face-of-brain-impairment,False,successful,2820.00,121,24,40,...,1,2012,12,30,4,2013,1,9,940.00,100.71
5,35,US,3500.00,annapolis-chamber-music-project,False,successful,3725.00,134,18,31,...,5,2015,4,30,159,2015,5,2,106.43,106.43


# export to csv

In [39]:
df.to_csv('data/kickstarter_preprocess.csv', index=False)